In [7]:
#pip install jupyter-dash==0.4

In [1]:
import dash
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output
import dash_html_components as html
import dash_bootstrap_components as dbc
from PIL import Image

C:\Users\Everydayuse\AppData\Local\Temp\ipykernel_18080\3887300031.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [131]:

banner_img = Image.open("assets/banner2.PNG")

link = html.A('View Github Repository', href="https://github.com/yourexodus/capstone_CDC")
 

banner_item = html.Div(
    [
        html.Div(
            html.Div(
                [
                    html.Div([
                        html.Img(src=banner_img, style={'width': '100%', 'height': '200px'})

                       # html.Img(src=banner_img, width=100%, height=100),               # using the pillow image variable
                        
                    ]),
                    html.Div(className="sidebar-wrapper"),
                ]
            ),
            className="sidebar",
        ),
        html.Div(
            html.Div(
                html.Div(className="container-fluid"),
                className="navbar navbar-expand-lg navbar-transparent navbar-absolute fixed-top ",
            ),
            className="main-panel",
        ),
    ]
)
banner_item.style = {'gridArea': "banner_item"}


layout = html.Div([link, banner_item])



app = dash.Dash(__name__)  # Initialize Dash app
app.layout = layout        #set layout
#app.run_server(mode='inline', height=100)  # Run the server
app.run_server(mode='inline', height=100, port=8052)  # Change the port number (e.g., 8051)

In [150]:
import pandas as pd
raw = pd.read_csv("https://archive.ics.uci.edu/static/public/891/data.csv")
raw.head()

,ID,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0,0,1,1,1,40,1,0,0,0,...,1,0,5,18,15,1,0,9,4,3
1,1,0,0,0,0,25,1,0,0,1,...,0,1,3,0,0,0,0,7,6,1
2,2,0,1,1,1,28,0,0,0,0,...,1,1,5,30,30,1,0,9,4,8
3,3,0,1,0,1,27,0,0,0,1,...,1,0,2,0,0,0,0,11,3,6
4,4,0,1,1,1,24,0,0,0,1,...,1,0,2,3,0,0,0,11,5,4


In [132]:
import dash
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output
import dash_html_components as html

In [133]:
import pandas as pd
summary = pd.read_csv("data/prepared/summary_data.csv")
summary = summary.reset_index()

In [134]:
used_columns = ["combined","count","percentage"]
df =summary[used_columns]
df = df.rename(columns={"combined":"generalhealth_type" ,"count":"total" }) 
df = df.reset_index()
#df["percentage"] =  df['percentage'].map('{:00.%}'.format)
df.head(3)

,index,generalhealth_type,total,percentage
0,0,Very good vs nondiabetic,82703,32.601309
1,1,Good vs nondiabetic,62189,24.514743
2,2,Excellent vs nondiabetic,44159,17.407364


#### Column info
 List of dictionaries:
    name - visible name;
    id - internal identification;
    type - "any", "numeric", "text";
    format - dictionary

In [135]:
df.columns[3:]

Index(['percentage'], dtype='object')

In [136]:
df.columns[:3]

Index(['index', 'generalhealth_type', 'total'], dtype='object')

In [137]:
columns=[]
columns = [{"name": "generalhealth_type", 
            "id": "gentype", "type": "text"}]
 
for name in df.columns[1:]:
    col_info = {
        "name": name,
         "id": name,
        "type": "numeric",
        "format": {'specifier': ','}
    }
    columns.append(col_info)
 
columns

[{'name': 'generalhealth_type', 'id': 'gentype', 'type': 'text'},
 {'name': 'generalhealth_type',
  'id': 'generalhealth_type',
  'type': 'numeric',
  'format': {'specifier': ','}},
 {'name': 'total',
  'id': 'total',
  'type': 'numeric',
  'format': {'specifier': ','}},
 {'name': 'percentage',
  'id': 'percentage',
  'type': 'numeric',
  'format': {'specifier': ','}}]

#### Convert columns to a list of dictionaries

In [138]:
#sort highest total to the top
data = df.sort_values("total",ascending=False).to_dict("records")
#view first 3 items in the list
data[:3]

[{'index': 0,
  'generalhealth_type': 'Very good vs nondiabetic',
  'total': 82703,
  'percentage': 32.6013087354147},
 {'index': 1,
  'generalhealth_type': 'Good vs nondiabetic',
  'total': 62189,
  'percentage': 24.51474298328603},
 {'index': 2,
  'generalhealth_type': 'Excellent vs nondiabetic',
  'total': 44159,
  'percentage': 17.407363607694734}]

#### Construct summary table

In [139]:

from dash_table import DataTable

In [140]:
def create_sum_table(group):
    
    used_columns = ["index","combined","count","percentage"]
    df =summary[used_columns]
    df = df.rename(columns={"combined":"generalhealth_type" ,"count":"total" }) 
    columns = []
    #columns = [{"name": "generalhealth_type", 
    #            "id": "gentype", "type": "text"}]    
    for name in df.columns:
        col_info = {
            "name": name,
             "id": name,
            "type": "text",
            "format": {'specifier': ','}
        }
        columns.append(col_info)

    print(columns) 

    data = df.sort_values("total",ascending=False).to_dict("records")
    return DataTable(
        id=f"{group}-table",           #sum-table
        columns=columns,
        data=data,
        active_cell={"row": 0, "column": 0},
        fixed_rows={"headers": True},
        sort_action="native",
        derived_virtual_data=data,
        style_table={
            "minHeight": "80vh",
            "height": "80vh",
            "overflowY": "scroll",
            "borderRadius": "0px 0px 10px 10px",
        },
        style_cell={
            "whiteSpace": "normal",
            "height": "auto",
            "fontFamily": "verdana",
        },
        style_header={
            "textAlign": "center",
            "fontSize": 14,
        },
        style_data={
            "fontSize": 12,
        },
        style_data_conditional=[
            {
                "if": {"column_id": "gentype"},
                "width": "420px",
                "textAlign": "left",
                "textDecoration": "underline",
                "cursor": "pointer",
            },
            
            {
                "if": {"column_id": "index"},
                "width": "70px",
                "textAlign": "left",
                "textDecoration": "underline",
                "cursor": "pointer",
            },
            {
                "if": {"row_index": "odd"}, 
                "backgroundColor": "#fafbfb"
            }
        ],
    )
 


In [141]:
summary[used_columns]

,combined,count,percentage
0,Very good vs nondiabetic,82703,32.601309
1,Good vs nondiabetic,62189,24.514743
2,Excellent vs nondiabetic,44159,17.407364
3,Fair vs nondiabetic,21780,8.585620
4,Good vs diabetic,13457,5.304715
5,Fair vs diabetic,9790,3.859193
6,Poor vs nondiabetic,7503,2.957663
7,Very good vs diabetic,6381,2.515374
8,Poor vs diabetic,4578,1.804636
9,Excellent vs diabetic,1140,0.449385


In [142]:
df = df.rename(columns={"combined":"generalhealth_type" ,"count":"total" })
df

,index,generalhealth_type,total,percentage
0,0,Very good vs nondiabetic,82703,32.601309
1,1,Good vs nondiabetic,62189,24.514743
2,2,Excellent vs nondiabetic,44159,17.407364
3,3,Fair vs nondiabetic,21780,8.585620
4,4,Good vs diabetic,13457,5.304715
5,5,Fair vs diabetic,9790,3.859193
6,6,Poor vs nondiabetic,7503,2.957663
7,7,Very good vs diabetic,6381,2.515374
8,8,Poor vs diabetic,4578,1.804636
9,9,Excellent vs diabetic,1140,0.449385


In [143]:
summary_table = create_sum_table(sum)

[{'name': 'index', 'id': 'index', 'type': 'text', 'format': {'specifier': ','}}, {'name': 'generalhealth_type', 'id': 'generalhealth_type', 'type': 'text', 'format': {'specifier': ','}}, {'name': 'total', 'id': 'total', 'type': 'text', 'format': {'specifier': ','}}, {'name': 'percentage', 'id': 'percentage', 'type': 'text', 'format': {'specifier': ','}}]


In [146]:
layout = html.Div([link, banner_item,   summary_table])


app = dash.Dash(__name__)  # Initialize Dash app
app.layout = layout        #set layout
#app.run_server(mode='inline', height=25)  # Run the server
app.run_server(mode='inline', height=100, port=8052)  # Change the port number (e.g., 8051)

In [145]:
import dash
from dash import dcc, html

app = dash.Dash(__name__)

options = [
    {'label': 'New York City', 'value': 'NYC'},
    {'label': 'Montreal', 'value': 'MTL'},
    {'label': 'San Francisco', 'value': 'SF'}
]

layout = html.Div([
    html.Div([
        dcc.Dropdown(
            id='my-dropdown1',
            options=options,
            placeholder="Select a city"
        ),
        dcc.Dropdown(
            id='my-dropdown2',
            options=options,
            placeholder="Select a city"
        ),
        dcc.Dropdown(
            id='my-dropdown3',
            options=options,
            placeholder="Select a city"
        )
    ], style={'display': 'flex'})
])

app = dash.Dash(__name__)  # Initialize Dash app
layout = html.Div([title,   summary_table])
app.run_server(mode='inline', height=25)  # Run the server


OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.

[2024-08-03 08:17:18,116] ERROR in app: Exception on /_alive_4804aadd-01f7-44d4-a9fb-ba14e2820618 [GET]
Traceback (most recent call last):
  File "C:\Users\Everydayuse\.conda\envs\capstoneCDC_venv\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Everydayuse\.conda\envs\capstoneCDC_venv\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Everydayuse\.conda\envs\capstoneCDC_venv\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Everydayuse\.conda\envs\capstoneCDC_venv\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Everydayuse\.conda\envs\capstoneCDC_venv\lib\site-packages\flask\app.py", line 1948, in full_dispatch_request
    rv = self.preprocess_request()
  File "C:\Users\Everydayuse\.conda\envs\capstoneCDC_venv\lib\site-packages\flas

In [ ]:
import dash
from dash import dcc, html

app = dash.Dash(__name__)

options = [
    {'label': 'New York City', 'value': 'NYC'},
    {'label': 'Montreal', 'value': 'MTL'},
    {'label': 'San Francisco', 'value': 'SF'}
]
menu_income:[
    {'label':'1 - Less than $10,000', 'value':1}, 
    {'label':'2 - Less than $15,000 ($10,000 to less than $15,000)', 'value':2},
    {'label':'3 - Less than $20,000 ($15,000 to less than $20,000)', 'value':3},
    {'label':'4 - Less than $25,000 ($20,000 to less than $25,000)', 'value':4},
    {'label':'5 - Less than $35,000 ($25,000 to less than $35,000)', 'value':5},
    {'label':'6 - Less than $50,000 ($35,000 to less than $50,000)', 'value':6},
    {'label':'7 - Less than $75,000 ($50,000 to less than $75,000)', 'value':7},
    {'label':'8 - $75,000 or more', 'value':8}
    ]
     
user_input =    html.Div([
        dcc.Dropdown(
            id='menu_income',
            options=options,
            placeholder="Please choose your income range."
            #value='NYC1'  # Initial value
        ),
        dcc.Dropdown(
            id='my-dropdown2',
            options=options,
            placeholder="Select a city"
            #value='NYC2'  # Initial value
        ),
        dcc.Dropdown(
            id='my-dropdown3',
            options=options,
            placeholder="Select a city"
            #value='NYC3'  # Initial value
        )        
        
    ])

layout = html.Div([title, user_input,  summary_table])

app = dash.Dash(__name__)  # Initialize Dash app
app.layout = layout        #set layout
app.run_server(mode='inline', height=25)  # Run the server

In [ ]:
layout = html.Div([title,   summary_table])


app = dash.Dash(__name__)  # Initialize Dash app
app.layout = layout        #set layout
app.run_server(mode='inline', height=25)  # Run the server